# Where to open a gym?

## Introduction

<p>Problem: A client is looking to open a gym in Ottawa and is looking for advice about where it should be located.</p>

<p>Ideally the location would be in an area that appears to be currently underserved by the availability of nearby gyms.
For this analysis, we will use the following information:</p>
<ul>
    <li>the number of gyms in each area in Ottawa using Forward Sortatation Areas (FSAs)</li>
    <li>the neighbourhood name(s) of each FSA (retrieved from Wikipedia)</li>
    <li>the population of each each FSA using the 2016 census</li>
    <li>the foursquare api to retrieve the current number of gyms per FSA</li>
</ul>
<p>Using the above information, we will cluster neighborhoods to find areas with a relatively high number of gyms and compare it to their population. We will use this data with the other areas to see if there is the potential to support another gym with their current population.</p>

# Data used in this project
<p>Wikipedia: <a href=https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K> list of postal codes of Canada starting with K </a> to get neighbourhood names for each Ottawa FSA.</p>
<p><a href=https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm> Statistics Canada </a> to get the population data for each FSA</p>    
<p>geopy: to get lat/long coordinates for Ottawa</p>
<p>pypostalcode: to get lat/long coordinates for each FSA</p>
<p>FourSquare Api: to get the number of gyms per FSA </p>

## Methodology

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim 

import requests # library to handle requests
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium==0.5.0 
import folium # map rendering library

!pip install pypostalcode
from pypostalcode import PostalCodeDatabase


print('Libraries imported.')

Libraries imported.


## Scraping Wikipedia for the Postal Codes in Ottawa

Import the lxml file from wikipedia

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_K').text
soup = BeautifulSoup(source, 'lxml')


Test to find table and first postal code

In [3]:
table = soup.find('table')
postcode = table.td.text
print(postcode)

K1AGovernment of CanadaOttawa and Gatineau offices (partly in QC)



Create a list for the postal code, city, and neighbourhood. 
Add them to a pandas dataframe.

In [4]:
postalCode = []
city = []
neighbourhood = []


In [5]:
for row in soup.find('table').find_all('tr'):
    items = row.find_all('td')
    for i in items:
        contents = i.text
        if "Not assigned" in contents: continue #remove any postal codes not assigned
        if "Ottawa" not in contents: continue #remove any postal codes not in Ottawa
        if "partly in QC" in contents: continue #remove any postal code partially in Quebec
        postalCode.append(contents[:3]) #first three characters are the postal code
        city.append(contents[3:9]) #next 6 characters are 'Ottawa'
        neighbourhood.append(contents[contents.find("(")+1:contents.find(")")].replace(' /', ',')) #Neighbouthoods are found between parentheses, change all / to ,

In [6]:
ottawa_df = pd.DataFrame({"PostalCode": postalCode,
                          "City": city,
                          "Neighbourhood":neighbourhood})
ottawa_df.head(10)

,PostalCode,City,Neighbourhood
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P..."
1,K4A,Ottawa,Fallingbrook
2,K1B,Ottawa,"Blackburn Hamlet, Pine View, Sheffield Glen"
3,K2B,Ottawa,"Britannia,Whitehaven, Bayshore, Pinecrest"
4,K4B,Ottawa,Navan
5,K1C,Ottawa,Orleans
6,K2C,Ottawa,"Queensway, Copeland Park, Central Park, Bel Ai..."
7,K4C,Ottawa,Cumberland
8,K1E,Ottawa,Queenswood
9,K2E,Ottawa,Eastern Nepean: Fisher Heights/ Parkwood Hills...


In [7]:
for nh in ottawa_df['Neighbourhood']:
    print(nh)

Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Orleans
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm,Pine Glen 
Riverview, Hawthorne, Canterbury, Hunt Club Park
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Alta Vista, Billings Bridge
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
Beacon Hill, Cyrville, Carson Grove
Barrhaven
Overbrook, Forbes, Manor Park, Viscount Alexander Park, Finter Quarries
Beaverbrook, South March
Vanier, McKay Lake area
Katimavik-Hazeldean, Glen Cairn
Rockcliffe Park, New Edinburgh
Bridlewood
Manotick
Lower Town, Byward Market, Sandy Hill, University of Ottawa
Downtown
Centretown
Greely
Dalhousie Ward
Fallowfield Village, Cedarhill Estates, Orchard Estates
The Glebe, Old Ottawa S

Print the number of rows in the dataframe

In [8]:
ottawa_df.shape

(40, 3)

# Import Population Data
<p> Accessed from <a href=https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/comprehensive.cfm> Statistics Canada </a></p>

In [9]:
pop_df = pd.read_csv('https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Tables/CompFile.cfm?Lang=Eng&T=1201&OFT=FULLCSV', encoding='utf-8')

In [10]:
pop_df.head()

,Geographic code,Geographic name,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,01,Canada,NaN,T,35151728.0,15412443.0,14072079.0
1,A0A,A0A,Newfoundland and Labrador,NaN,46587.0,26155.0,19426.0
2,A0B,A0B,Newfoundland and Labrador,NaN,19792.0,13658.0,8792.0
3,A0C,A0C,Newfoundland and Labrador,NaN,12587.0,8010.0,5606.0
4,A0E,A0E,Newfoundland and Labrador,NaN,22294.0,12293.0,9603.0


In [11]:
pop_df.rename(columns = {"Geographic name":"PostalCode"}, inplace = True)
pop_df.head()

,Geographic code,PostalCode,Province or territory,"Incompletely enumerated Indian reserves and Indian settlements, 2016","Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,01,Canada,NaN,T,35151728.0,15412443.0,14072079.0
1,A0A,A0A,Newfoundland and Labrador,NaN,46587.0,26155.0,19426.0
2,A0B,A0B,Newfoundland and Labrador,NaN,19792.0,13658.0,8792.0
3,A0C,A0C,Newfoundland and Labrador,NaN,12587.0,8010.0,5606.0
4,A0E,A0E,Newfoundland and Labrador,NaN,22294.0,12293.0,9603.0


In [12]:
unwcol = ['Geographic code','Province or territory','Incompletely enumerated Indian reserves and Indian settlements, 2016']
pop_df.drop(unwcol, inplace=True, axis=1)
pop_df.head()

,PostalCode,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,Canada,35151728.0,15412443.0,14072079.0
1,A0A,46587.0,26155.0,19426.0
2,A0B,19792.0,13658.0,8792.0
3,A0C,12587.0,8010.0,5606.0
4,A0E,22294.0,12293.0,9603.0


In [13]:
ottawapop_df = ottawa_df.merge(pop_df, on="PostalCode", how="left")
ottawapop_df.head()

,PostalCode,City,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0
1,K4A,Ottawa,Fallingbrook,54524.0,18677.0,18544.0
2,K1B,Ottawa,"Blackburn Hamlet, Pine View, Sheffield Glen",17110.0,6826.0,6611.0
3,K2B,Ottawa,"Britannia,Whitehaven, Bayshore, Pinecrest",32260.0,15640.0,14719.0
4,K4B,Ottawa,Navan,4793.0,1771.0,1732.0


# Import FSA Coordinates from pypostalcode

In [14]:
pcdb = PostalCodeDatabase()
longitude = []
latitude = []
print(postalCode)

['K2A', 'K4A', 'K1B', 'K2B', 'K4B', 'K1C', 'K2C', 'K4C', 'K1E', 'K2E', 'K1G', 'K2G', 'K1H', 'K2H', 'K1J', 'K2J', 'K1K', 'K2K', 'K1L', 'K2L', 'K1M', 'K2M', 'K4M', 'K1N', 'K1P', 'K2P', 'K4P', 'K1R', 'K2R', 'K1S', 'K2S', 'K1T', 'K2T', 'K1V', 'K2V', 'K1W', 'K2W', 'K1X', 'K1Y', 'K1Z']


In [15]:
for pc in postalCode:
    location = pcdb[pc]
    longitude.append(location.longitude)
    latitude.append(location.latitude)

In [16]:
ottawa_postal_codes_df = pd.DataFrame({"PostalCode": postalCode,
                          "Longitude": longitude,
                          "Latitude": latitude})
ottawa_postal_codes_df.head(10)

,PostalCode,Longitude,Latitude
0,K2A,-75.7632,45.3778
1,K4A,-75.4835,45.4769
2,K1B,-75.5624,45.4325
3,K2B,-75.7888,45.3679
4,K4B,-75.4288,45.4251
5,K1C,-75.5237,45.4805
6,K2C,-75.7523,45.3594
7,K4C,-75.4108,45.5177
8,K1E,-75.5199,45.4882
9,K2E,-75.7209,45.3353


In [17]:
ottawapopcoor_df = ottawapop_df.merge(ottawa_postal_codes_df, on="PostalCode", how="left")
ottawapopcoor_df.head()

,PostalCode,City,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Longitude,Latitude
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0,-75.7632,45.3778
1,K4A,Ottawa,Fallingbrook,54524.0,18677.0,18544.0,-75.4835,45.4769
2,K1B,Ottawa,"Blackburn Hamlet, Pine View, Sheffield Glen",17110.0,6826.0,6611.0,-75.5624,45.4325
3,K2B,Ottawa,"Britannia,Whitehaven, Bayshore, Pinecrest",32260.0,15640.0,14719.0,-75.7888,45.3679
4,K4B,Ottawa,Navan,4793.0,1771.0,1732.0,-75.4288,45.4251


In [18]:
print('The dataframe has {} neighbourhoods.'.format(
        ottawapopcoor_df.shape[0])
    )

The dataframe has 40 neighbourhoods.


## Map of the neighbourhoods in Ottawa

In [19]:
address = 'Ottawa, ON'

geolocator = Nominatim(user_agent="on_explorer")
oloc = geolocator.geocode(address)
olat = location.latitude
olong = location.longitude
print('The geograpical coordinates of Ottawa are {}, {}.'.format(olat, olong))

The geograpical coordinates of Ottawa are 45.3956, -75.7462.


Create a map of Ottawa with neighbourhoods superimposed on top.

In [60]:
# create map of Ottawa using latitude and longitude values
map_ottawa = folium.Map(location=[olat, olong], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(ottawapopcoor_df['Latitude'], ottawapopcoor_df['Longitude'], ottawapopcoor_df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ottawa)  
    
map_ottawa

# Foursquare 

<p>We are looking for these specific venue categories:</p>
<ul>
    <li>College Gym: 4bf58dd8d48988d1b2941735</li>
    <li>Gym / Fitness Center: 4bf58dd8d48988d175941735</li>
    <li>Gym: 4bf58dd8d48988d176941735</li>
   </ul>

In [21]:
# The code was removed by Watson Studio for sharing.

In [22]:
LIMIT = 100
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    categoryID = '4bf58dd8d48988d1b2941735,4bf58dd8d48988d175941735,4bf58dd8d48988d176941735'
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            categoryID)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']

        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
ottawa_gyms= getNearbyVenues(names=ottawapopcoor_df['Neighbourhood'],
                                   latitudes=ottawapopcoor_df['Latitude'],
                                   longitudes=ottawapopcoor_df['Longitude']
                                  )


Highland Park, McKellar Park,Westboro,Glabar Park,Carlingwood
Fallingbrook
Blackburn Hamlet, Pine View, Sheffield Glen
Britannia,Whitehaven, Bayshore, Pinecrest
Navan
Orleans
Queensway, Copeland Park, Central Park, Bel Air,Carleton Heights
Cumberland
Queenswood
Eastern Nepean: Fisher Heights/ Parkwood Hills, Borden Farm,Pine Glen 
Riverview, Hawthorne, Canterbury, Hunt Club Park
Centrepointe, Meadowlands, City View, Craig Henry, Tangelwood, Grenfell Glen, Davidson Heights
Alta Vista, Billings Bridge
Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach
Beacon Hill, Cyrville, Carson Grove
Barrhaven
Overbrook, Forbes, Manor Park, Viscount Alexander Park, Finter Quarries
Beaverbrook, South March
Vanier, McKay Lake area
Katimavik-Hazeldean, Glen Cairn
Rockcliffe Park, New Edinburgh
Bridlewood
Manotick
Lower Town, Byward Market, Sandy Hill, University of Ottawa
Downtown
Centretown
Greely
Dalhousie Ward
Fallowfield Village, Cedarhill Estates, Orchard Estates
The Glebe, Old Ottawa S

In [24]:
ottawa_gyms.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Soloway Jewish Community Centre,45.375805,-75.752173,Gym
1,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Anytime Fitness,45.375510,-75.757810,Gym / Fitness Center
2,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Inside Out Studio Barre,45.397982,-75.740246,Dance Studio
3,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,GoodLife Fitness Ottawa Baseline and Woodroffe,45.351950,-75.759514,Gym
4,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Fit4Less,45.361929,-75.735351,Gym


In [25]:
ottawa_gyms.shape

(499, 7)

In [26]:
centretown_gyms = ottawa_gyms.loc[ottawa_gyms['Neighbourhood'] == 'Centretown']
centretown_gyms.head(15)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
273,Centretown,45.4129,-75.6901,GoodLife Fitness Ottawa Lansdowne Park,45.399084,-75.683930,Gym
274,Centretown,45.4129,-75.6901,Minto Sports Complex | Complexe sportif Minto ...,45.420553,-75.678318,College Gym
275,Centretown,45.4129,-75.6901,Downtown YMCA-YWCA,45.420687,-75.699806,Gym
276,Centretown,45.4129,-75.6901,Taggart Family YMCA,45.411268,-75.689711,Gym
277,Centretown,45.4129,-75.6901,GoodLife Fitness Ottawa Queen and Bank,45.421146,-75.700422,Gym
278,Centretown,45.4129,-75.6901,yogatown,45.403483,-75.711239,Yoga Studio
279,Centretown,45.4129,-75.6901,Plant Recreation Centre,45.407927,-75.714710,Recreation Center
280,Centretown,45.4129,-75.6901,Rama Lotus Yoga Centre,45.413512,-75.692819,Yoga Studio
281,Centretown,45.4129,-75.6901,Pure Yoga Centretown,45.414221,-75.695167,Yoga Studio
282,Centretown,45.4129,-75.6901,Trueform Fitness,45.413490,-75.694840,Gym


<p>It looks like there are some venue types that we don't want such as 'Gym Pool' and 'Yoga Studio'.</p>
Let's limit it to 'Gym', 'Gym / Fitness Center' and 'College Gym'

In [27]:
ottawa_gym_fit = ottawa_gyms.loc[ottawa_gyms['Venue Category'].isin(['Gym','Gym / Fitness Center','College Gym'])]
ottawa_gym_fit.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Soloway Jewish Community Centre,45.375805,-75.752173,Gym
1,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Anytime Fitness,45.375510,-75.757810,Gym / Fitness Center
3,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,GoodLife Fitness Ottawa Baseline and Woodroffe,45.351950,-75.759514,Gym
4,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Fit4Less,45.361929,-75.735351,Gym
5,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Ottawa Gymnastics Centre,45.394251,-75.752520,Gym


In [28]:
ottawa_gym_fit.shape

(344, 7)

In [29]:
ottawa_gym_fit.groupby(['Neighbourhood']).count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Alta Vista, Billings Bridge",10,10,10,10,10,10
Barrhaven,4,4,4,4,4,4
"Beacon Hill, Cyrville, Carson Grove",8,8,8,8,8,8
"Beaverbrook, South March",6,6,6,6,6,6
"Bells Corners, Arlington Woods/Redwood, Qualicum, Crystal Beach",3,3,3,3,3,3
"Blackburn Hamlet, Pine View, Sheffield Glen",2,2,2,2,2,2
"Blossom Park, Greenboro, Leitrim, Findlay Creek",2,2,2,2,2,2
Bridlewood,6,6,6,6,6,6
"Britannia,Whitehaven, Bayshore, Pinecrest",9,9,9,9,9,9


In [30]:
ottawa_gym_fit.to_csv('OttawaGyms.csv', encoding='utf-8', index=False)
print("Saved!")

Saved!


In [31]:
ottawa_gym_fit['Venue'].value_counts()

Anytime Fitness                                            39
Fit4Less                                                   12
Snap Fitness                                               10
Movati Athletic                                             7
University of Ottawa Gym                                    6
Minto Sports Complex | Complexe sportif Minto - uOttawa     6
Therien Martial Arts                                        6
Free Form Fitness                                           6
Starr Gymnastics                                            5
Greco Lean + Fit                                            5
CrossFit Bytown                                             5
Champagne Fitness Centre                                    5
Movati Athletic Trainyards                                  5
Taggart Family YMCA                                         5
Custom strength                                             5
Gladstone Sports and Health Center                          5
Hintonbu

<p>With the proximity of neighbourhoods, some gyms have been counted multiple times, such as 'Taggart Family YMCA'. </p>
<p>We will clean this up based on Latitude and Longitude. The steps we will take are:</p>
<ol>
    <li>Dropping any rows that have the same 'Venue Latitude' and 'Venue Longitude' into a new dataframe gymd (gym dropped)</li>
    <li>Making a new dataframe named duplicates that takes all the rows in the ottawa_gym_fit dataframe that are not in the gymd dataframe</li>
    <ul style="list-style-type: lower-alpha;">
        <li>This extracts all the duplicate gyms</li>
    </ul>
    <li>Comparing the Venue lat/long to the Neighbourhood lat/long to find the smallest distance</li>
    <li>Dropping all rows that are not the smallest distance into the dataframe closest_neighbourhood</li>
    <li>Concatenating the cosest_neighbourhood and gymd dataframes to cleaned_df to have a dataframe with all the gyms with no duplicates</li>
 </ol>

In [32]:
gymd = ottawa_gym_fit.drop_duplicates(subset=['Venue Latitude', 'Venue Longitude'], keep=False)
gymd.sort_values(['Venue'])

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
22,Fallingbrook,45.4769,-75.4835,9Round Fitness,45.468555,-75.455414,Gym / Fitness Center
448,"Chapel Hill South, Blackburn",45.4360,-75.5471,9Round Fitness,45.450365,-75.518733,Gym / Fitness Center
134,Barrhaven,45.2882,-75.7566,Anytime Fitness,45.270120,-75.742260,Gym / Fitness Center
24,Fallingbrook,45.4769,-75.4835,Anytime Fitness,45.451280,-75.484250,Gym / Fitness Center
27,"Blackburn Hamlet, Pine View, Sheffield Glen",45.4325,-75.5624,Anytime Fitness,45.447030,-75.593510,Gym / Fitness Center
210,Manotick,45.2289,-75.6817,Anytime Fitness,45.223410,-75.684480,Gym / Fitness Center
81,Eastern Nepean: Fisher Heights/ Parkwood Hills...,45.3353,-75.7209,Anytime Fitness,45.334420,-75.691660,Gym / Fitness Center
43,Orleans,45.4805,-75.5237,Bob MacQuarrie Recreation Complex,45.466270,-75.544204,Gym / Fitness Center
119,"Bells Corners, Arlington Woods/Redwood, Qualic...",45.3155,-75.8370,Cure Is To Laugh,45.313291,-75.831366,Gym
75,Eastern Nepean: Fisher Heights/ Parkwood Hills...,45.3353,-75.7209,Dynamo Academy,45.323279,-75.721879,Gym / Fitness Center


In [33]:
duplicates = ottawa_gym_fit[~(ottawa_gym_fit['Venue Latitude'].isin(gymd['Venue Latitude']) & ottawa_gym_fit['Venue Longitude'].isin(gymd['Venue Longitude']) )].dropna().reset_index(drop=True)

In [34]:
duplicates.sort_values(['Venue Latitude']).reset_index(drop = True).head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,Greco Lean+fit,45.266056,-75.771768,Gym
1,Barrhaven,45.2882,-75.7566,Greco Lean+fit,45.266056,-75.771768,Gym
2,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center
3,Barrhaven,45.2882,-75.7566,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center
4,Stittsville,45.2573,-75.9153,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center
5,"Terry Fox, Palladium",45.3018,-75.9081,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center
6,Bridlewood,45.2884,-75.8648,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym
7,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym
8,"Lower Town, Byward Market, Sandy Hill, Univers...",45.3176,-75.8950,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym
9,"Terry Fox, Palladium",45.3018,-75.9081,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym


In [35]:
duplicates['Distance from Neighbourhood'] = (duplicates['Neighbourhood Latitude'] - duplicates['Venue Latitude']).abs()+(duplicates['Neighbourhood Longitude'] - duplicates['Venue Longitude']).abs()

In [36]:
duplicates.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance from Neighbourhood
0,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Soloway Jewish Community Centre,45.375805,-75.752173,Gym,0.013023
1,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Anytime Fitness,45.375510,-75.757810,Gym / Fitness Center,0.007680
2,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,GoodLife Fitness Ottawa Baseline and Woodroffe,45.351950,-75.759514,Gym,0.029536
3,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Fit4Less,45.361929,-75.735351,Gym,0.043720
4,"Highland Park, McKellar Park,Westboro,Glabar P...",45.3778,-75.7632,Ottawa Gymnastics Centre,45.394251,-75.752520,Gym,0.027131


In [37]:
duplicates_sorted = duplicates.sort_values(by = ['Venue Latitude', 'Distance from Neighbourhood']).reset_index(drop = True)
duplicates_sorted.head(10)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance from Neighbourhood
0,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,Greco Lean+fit,45.266056,-75.771768,Gym,0.029976
1,Barrhaven,45.2882,-75.7566,Greco Lean+fit,45.266056,-75.771768,Gym,0.037312
2,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center,0.017522
3,Barrhaven,45.2882,-75.7566,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center,0.044400
4,"Terry Fox, Palladium",45.3018,-75.9081,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center,0.022432
5,Stittsville,45.2573,-75.9153,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center,0.029268
6,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym,0.022393
7,"Terry Fox, Palladium",45.3018,-75.9081,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym,0.024023
8,"Lower Town, Byward Market, Sandy Hill, Univers...",45.3176,-75.8950,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym,0.026723
9,Bridlewood,45.2884,-75.8648,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym,0.032677


In [38]:
closest_neighbourhood = duplicates_sorted.drop_duplicates(subset=['Venue Latitude', 'Venue Longitude']).reset_index(drop = True)
closest_neighbourhood.head(10)

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Distance from Neighbourhood
0,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,Greco Lean+fit,45.266056,-75.771768,Gym,0.029976
1,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center,0.017522
2,"Terry Fox, Palladium",45.3018,-75.9081,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center,0.022432
3,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym,0.022393
4,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,Anytime Fitness,45.299670,-75.887690,Gym / Fitness Center,0.016720
5,"Terry Fox, Palladium",45.3018,-75.9081,Movati Athletic,45.300576,-75.916076,Gym / Fitness Center,0.009199
6,"Terry Fox, Palladium",45.3018,-75.9081,Fitness Depot,45.310826,-75.906558,Gym / Fitness Center,0.010568
7,Marchwood,45.3121,-75.9217,"iLoveKickboxing - Kanata-Centrum, ON",45.311443,-75.914271,Gym,0.008086
8,"Centrepointe, Meadowlands, City View, Craig He...",45.3286,-75.7703,Nepean Sportsplex,45.326867,-75.746168,Gym,0.025865
9,"Beaverbrook, South March",45.3339,-75.9098,Free Form Fitness,45.338464,-75.904796,Gym / Fitness Center,0.009568


In [39]:
closest_neighbourhood.drop(['Distance from Neighbourhood'], axis =1, inplace=True)
closest_neighbourhood.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,Greco Lean+fit,45.266056,-75.771768,Gym
1,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center
2,"Terry Fox, Palladium",45.3018,-75.9081,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center
3,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym
4,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,Anytime Fitness,45.299670,-75.887690,Gym / Fitness Center


In [40]:
cleaned_df = pd.concat([closest_neighbourhood, gymd])
cleaned_df.sort_values(["Venue Latitude"]).head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
210,Manotick,45.2289,-75.6817,Anytime Fitness,45.223410,-75.684480,Gym / Fitness Center
415,Stittsville,45.2573,-75.9153,Goulbourn Recreation Complex,45.263501,-75.907433,Gym
0,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,Greco Lean+fit,45.266056,-75.771768,Gym
133,Barrhaven,45.2882,-75.7566,GoodLife Fitness Barrhaven Chapman Mills Marke...,45.268051,-75.743460,Gym
416,Stittsville,45.2573,-75.9153,GoodLife Fitness Stittsville Corners,45.268645,-75.940997,Gym


In [41]:
cleaned_df['Venue'].value_counts()

Anytime Fitness                                            14
Fit4Less                                                    6
GoodLife Fitness                                            4
Planet Fitness                                              3
Starr Gymnastics                                            3
Movati Athletic                                             3
Snap Fitness                                                3
9Round Fitness                                              3
Free Form Fitness                                           2
Ruddy Family YMCA-YWCA                                      1
Physical Recreation Centre                                  1
Pinecrest Recreation Complex                                1
Ray Friel Recreation Complex                                1
GoodLife Fitness Kanata Hazeldean and Castlefrank           1
Ottawa Athletic Club                                        1
The Fitness Fusion Studio                                   1
Nepean-C

We will now count the number of gyms per neighbourhood

In [42]:
cleaned_df["Number of Gyms"] = cleaned_df.groupby('Neighbourhood')['Neighbourhood'].transform('count')
cleaned_df

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Number of Gyms
0,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,Greco Lean+fit,45.266056,-75.771768,Gym,2
1,"Fallowfield Village, Cedarhill Estates, Orchar...",45.2776,-75.7902,9Round Fitness,45.268739,-75.781539,Gym / Fitness Center,2
2,"Terry Fox, Palladium",45.3018,-75.9081,Olympia Gymnastics,45.280687,-75.909419,Gym / Fitness Center,3
3,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,GoodLife Fitness Kanata Hazeldean and Castlefrank,45.296092,-75.889785,Gym,2
4,"Katimavik-Hazeldean, Glen Cairn",45.3125,-75.8838,Anytime Fitness,45.299670,-75.887690,Gym / Fitness Center,2
5,"Terry Fox, Palladium",45.3018,-75.9081,Movati Athletic,45.300576,-75.916076,Gym / Fitness Center,3
6,"Terry Fox, Palladium",45.3018,-75.9081,Fitness Depot,45.310826,-75.906558,Gym / Fitness Center,3
7,Marchwood,45.3121,-75.9217,"iLoveKickboxing - Kanata-Centrum, ON",45.311443,-75.914271,Gym,1
8,"Centrepointe, Meadowlands, City View, Craig He...",45.3286,-75.7703,Nepean Sportsplex,45.326867,-75.746168,Gym,2
9,"Beaverbrook, South March",45.3339,-75.9098,Free Form Fitness,45.338464,-75.904796,Gym / Fitness Center,4


In [43]:
cleaned_df.shape

(126, 8)

In [61]:
# create map of Ottawa with the gyms superimposed
map_ottawa_gyms = folium.Map(location=[olat, olong], zoom_start=10)

# add markers to map
for lat, lng, venue in zip(cleaned_df['Venue Latitude'], cleaned_df['Venue Longitude'], cleaned_df['Venue']):
    label = '{}'.format(venue)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=1,
        popup=label,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ottawa_gyms)  
    
map_ottawa_gyms

Add in the population data and remove the unecessary information:

In [45]:
gym_pop = ottawapop_df.merge(cleaned_df, on="Neighbourhood", how="left")
gym_pop.head()

,PostalCode,City,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Number of Gyms
0,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0,45.3778,-75.7632,Premier Fitness,45.372781,-75.769122,Gym,4.0
1,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0,45.3778,-75.7632,Carlingwood YMCA-YWCA,45.373569,-75.769207,Gym / Fitness Center,4.0
2,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0,45.3778,-75.7632,Anytime Fitness,45.375510,-75.757810,Gym / Fitness Center,4.0
3,K2A,Ottawa,"Highland Park, McKellar Park,Westboro,Glabar P...",16790.0,7433.0,7205.0,45.3778,-75.7632,Soloway Jewish Community Centre,45.375805,-75.752173,Gym,4.0
4,K4A,Ottawa,Fallingbrook,54524.0,18677.0,18544.0,45.4769,-75.4835,Ray Friel Recreation Complex,45.471871,-75.492555,Gym,10.0


In [46]:
pd.options.display.float_format = '{:,.0f}'.format
unwanted_columns = ['City','Neighbourhood Latitude','Neighbourhood Longitude','Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
gym_pop.drop(unwanted_columns , inplace=True, axis=1)
gym_pop.head()

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4
1,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4
2,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4
3,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4
4,K4A,Fallingbrook,"54,524","18,677","18,544",10


In [47]:
gym_pop = gym_pop.drop_duplicates(subset=['Neighbourhood']).reset_index(drop=True)
gym_pop.head()

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4
1,K4A,Fallingbrook,"54,524","18,677","18,544",10
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen","17,110","6,826","6,611",2
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest","32,260","15,640","14,719",3
4,K4B,Navan,"4,793","1,771","1,732",nan


Add a number of people per gym column

In [48]:
gym_pop['People per Gym'] = gym_pop['Population, 2016']/gym_pop['Number of Gyms']
gym_pop.head()

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms,People per Gym
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4,"4,198"
1,K4A,Fallingbrook,"54,524","18,677","18,544",10,"5,452"
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen","17,110","6,826","6,611",2,"8,555"
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest","32,260","15,640","14,719",3,"10,753"
4,K4B,Navan,"4,793","1,771","1,732",nan,nan


In [49]:
gym_pop['Occupied Dwellings per Gym'] = gym_pop['Private dwellings occupied by usual residents, 2016']/gym_pop['Number of Gyms']
gym_pop.head()

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms,People per Gym,Occupied Dwellings per Gym
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4,"4,198","1,801"
1,K4A,Fallingbrook,"54,524","18,677","18,544",10,"5,452","1,854"
2,K1B,"Blackburn Hamlet, Pine View, Sheffield Glen","17,110","6,826","6,611",2,"8,555","3,306"
3,K2B,"Britannia,Whitehaven, Bayshore, Pinecrest","32,260","15,640","14,719",3,"10,753","4,906"
4,K4B,Navan,"4,793","1,771","1,732",nan,nan,nan


In [50]:
gym_pop.sort_values(by = ['Number of Gyms'], ascending = False)

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms,People per Gym,Occupied Dwellings per Gym
25,K2P,Centretown,"17,603","13,302","11,706",14,"1,257",836
24,K1P,Downtown,340,469,230,13,26,18
1,K4A,Fallingbrook,"54,524","18,677","18,544",10,"5,452","1,854"
9,K2E,Eastern Nepean: Fisher Heights/ Parkwood Hills...,"19,034","8,424","7,976",9,"2,115",886
27,K1R,Dalhousie Ward,"18,730","12,095","10,518",6,"3,122","1,753"
5,K1C,Orleans,"37,662","14,338","14,172",6,"6,277","2,362"
6,K2C,"Queensway, Copeland Park, Central Park, Bel Ai...","27,941","12,901","11,863",6,"4,657","1,977"
17,K2K,"Beaverbrook, South March","24,431","9,326","8,921",4,"6,108","2,230"
21,K2M,Bridlewood,"27,028","9,308","9,226",4,"6,757","2,306"
0,K2A,"Highland Park, McKellar Park,Westboro,Glabar P...","16,790","7,433","7,205",4,"4,198","1,801"


In [51]:
avg_pop_gym = int(gym_pop[["People per Gym"]].mean())
print('There are {} people per gym in the neighbourhoods that already have a gym'.format(avg_pop_gym))

There are 8471 people per gym in the neighbourhoods that already have a gym


In [52]:
num_gyms = int(gym_pop[['Number of Gyms']].sum())
print('There are {} gyms in the city of Ottawa'.format(num_gyms))

There are 126 gyms in the city of Ottawa


In [53]:
ott_pop = int(gym_pop[['Population, 2016']].sum())
print('The total population of Ottawa was {} in 2016.'.format(ott_pop))

The total population of Ottawa was 881580 in 2016.


In [54]:
ottawa_pop_gym = int(ott_pop/num_gyms)

print('The are {} people per gym in all of Ottawa.'.format(str(ottawa_pop_gym)))

The are 6996 people per gym in all of Ottawa.


In [55]:
gym_candidates = gym_pop.loc[(gym_pop['People per Gym'] > avg_pop_gym*2)].sort_values(by = ['People per Gym'], ascending = False)
gym_candidates.head()

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms,People per Gym,Occupied Dwellings per Gym
15,K2J,Barrhaven,"68,963","23,593","23,145",2,"34,482","11,572"
11,K2G,"Centrepointe, Meadowlands, City View, Craig He...","49,222","18,145","17,506",2,"24,611","8,753"
31,K1T,"Blossom Park, Greenboro, Leitrim, Findlay Creek","35,432","12,557","12,219",2,"17,716","6,110"


In [79]:
num_gym_candidates = len(gym_candidates.index)
print('There are {} potential neighbourhoods that already have a gym.'.format(num_gym_candidates))

There are 3 potential neighbourhoods that already have a gym.


In [57]:
nan_gyms = gym_pop[pd.isnull(gym_pop).any(axis=1)].sort_values(by = ['Population, 2016'], ascending = False)
nan_gym_candidates = nan_gyms.loc[(nan_gyms['Population, 2016'] > avg_pop_gym)].sort_values(by = ['Population, 2016'], ascending = False)
nan_gym_candidates

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms,People per Gym,Occupied Dwellings per Gym
33,K1V,"Heron Gate, Heron Park, Riverside Park, Hunt C...","54,835","22,427","21,048",nan,nan,nan
23,K1N,"Lower Town, Byward Market, Sandy Hill, Univers...","25,063","16,708","13,262",nan,nan,nan
18,K1L,"Vanier, McKay Lake area","17,021","10,452","9,369",nan,nan,nan
8,K1E,Queenswood,"14,803","5,815","5,742",nan,nan,nan
26,K4P,Greely,"9,821","3,336","3,273",nan,nan,nan
36,K2W,North March,"8,660","2,624","2,603",nan,nan,nan


In [77]:
num_nan_candidates =len(nan_gym_candidates.index)
print("There are {} potential neighbourhoods that don't already have a gym.".format(num_nan_candidates))

There are 6 potential neighbourhoods that don't already have a gym.


In [69]:
all_candidates = pd.concat([nan_gym_candidates, gym_candidates])
all_candidates

,PostalCode,Neighbourhood,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016",Number of Gyms,People per Gym,Occupied Dwellings per Gym
33,K1V,"Heron Gate, Heron Park, Riverside Park, Hunt C...","54,835","22,427","21,048",nan,nan,nan
23,K1N,"Lower Town, Byward Market, Sandy Hill, Univers...","25,063","16,708","13,262",nan,nan,nan
18,K1L,"Vanier, McKay Lake area","17,021","10,452","9,369",nan,nan,nan
8,K1E,Queenswood,"14,803","5,815","5,742",nan,nan,nan
26,K4P,Greely,"9,821","3,336","3,273",nan,nan,nan
36,K2W,North March,"8,660","2,624","2,603",nan,nan,nan
15,K2J,Barrhaven,"68,963","23,593","23,145",2,"34,482","11,572"
11,K2G,"Centrepointe, Meadowlands, City View, Craig He...","49,222","18,145","17,506",2,"24,611","8,753"
31,K1T,"Blossom Park, Greenboro, Leitrim, Findlay Creek","35,432","12,557","12,219",2,"17,716","6,110"


In [74]:
del_columns= ['Total private dwellings, 2016','Private dwellings occupied by usual residents, 2016','Occupied Dwellings per Gym']
all_candidates.drop(del_columns, inplace=True, axis=1)

In [76]:
all_candidates.sort_values(by = ['Population, 2016'], ascending = False).reset_index(drop=True)

,PostalCode,Neighbourhood,"Population, 2016",Number of Gyms,People per Gym
0,K2J,Barrhaven,"68,963",2,"34,482"
1,K1V,"Heron Gate, Heron Park, Riverside Park, Hunt C...","54,835",nan,nan
2,K2G,"Centrepointe, Meadowlands, City View, Craig He...","49,222",2,"24,611"
3,K1T,"Blossom Park, Greenboro, Leitrim, Findlay Creek","35,432",2,"17,716"
4,K1N,"Lower Town, Byward Market, Sandy Hill, Univers...","25,063",nan,nan
5,K1L,"Vanier, McKay Lake area","17,021",nan,nan
6,K1E,Queenswood,"14,803",nan,nan
7,K4P,Greely,"9,821",nan,nan
8,K2W,North March,"8,660",nan,nan


In [78]:
num_all_candidates = len(all_candidates.index)
print("There is a total of {} potential neighbourhoods that could support a gym.".format(num_all_candidates))

There is a total of 9 potential neighbourhoods that could support a gym.


## Results and Discussion

<p>Our analysis shows that there is 126 gyms in the Ottawa area. A total of 40 Forward Sortatation Areas (FSAs) or neighbourhoods were used to group the gyms, and then compared to their population in 2016.</p>
<p>The gyms are spread out thoughout the city, with a high density of gyms in the Downtown, Centretown, Fallingbrook, and Eastern Nepean neighbourhoods. On average there are 6966 people per gym in the whole city of Ottawa, but when you take only take into account the neighbourhoods that already include a gym that number changes to 8471 people per gym.</p>
<p>To find the neighbourhoods that could potentially support a gym, we looked for neighbourhoods that could support an additional 8471 people per gym. If the neighbourhood already had at least one gym, the neighbourhood would have to have a 'People per Gym' greater than twice 8471. If the neighbourhood did not already have a gym, the neighbourhood would have to have a population greater than 8471.</p> 
<p>9 neighbourhoods were found to match this criteria, but this does not imply that all those neighbourhoods are able to support a new gym. It is possible that a gym does not exist in those neighbourhoods because they are not able to support them. The recommended neighbourhoods should only be considered as a starting point for a more detailed analysis which could eventually result in a location which not only has no nearby competition but also meets other relevant conditions.</p>

# Conclusion

<p>The purpose of this project was to identify possible areas in Ottawa to open a new gym. In order to aid the stakeholders in narrowing down their search for a location, Foursquare was used to find the density of gyms throughout the different FSAs in the city, and then compared to the population of those areas. Potentail neighbourhoods were found and should be used as a starting point for final exploration by the stakeholders.</p>
<p>Final decission on optimal gym location will be made by stakeholders based on specific characteristics of neighbourhoods, taking into consideration additional factors like the average income of each location, availability of parking, proximity to public transit, real estate availability, prices, social and economic dynamics of every neighborhood etc.</p>